# Universidad del Rosario
# Procesamiento de Lenguaje Natural
# Caso de estudio 2 corte 2
# Fabián Sánchez

# Caso corte 2

- Para entregar antes del día martes 18 de abril a las 18:00. En grupos de dos integrantes

- Cargar el .pynb en la carpeta de las aulas virtuales

- Los avances del caso se realizará el día 12 de abril en clase y se evaluarán los avances y el trabajo en la clase.

- El trabajo completo se cargará de acuerdo a las anteriores instrucciones.

Considere el siguiente conjunto de datos

In [1]:
url= "https://raw.githubusercontent.com/Fabian830348/Bases_Datos/master/base_LDA.json"

## Librerias

In [2]:
import pandas as pd
from langdetect import detect
import spacy
import re

In [3]:
#Este es utilizado para español
nlp_es = spacy.load('es_core_news_sm')
# Este es utilizado oara ingles
nlp_en = spacy.load("en_core_web_sm")

# PUNTO 1

* Considere el siguiente conjunto de documentos ***obtener los temas (Topics) de una serie de artículos (noticias)*** que está etiquetados con un tema (el corpus tiene 20 temas).

- El fichero en formato .Json contiene una serie de documento en los que le asigna una temática.

- Cada elemento del Json contiene:

    - **content**: Contenido del artículo
    - **target**: Identificador del target
    - **target_names**: Nombre del target


* ***NOTA***: *Ya que vamos a realizar un ejercicio de aprendizjae no supervisado, no debemos tener en cuentas los temas en los que alguien (un humano experto) ha clasificado estos textos. La idea del ejercicio es obtener los temas distintos de los que hablan los artículos (***Clusterizar artículos***). En este sentido es útil saber a priori el número de temas distintos que puede tener, pero en ningún caso el target de los artículos entraria al algoritmo de aprendizaje*

* Para realizar este ejercicio se seguirán los siguientes pasos

5. Crear del diccionario y la bolsa de palabras

    5.1 Corpus tokenizado: "*documents_tok*"

    5.2 Diccionario: "*dictionary*"

    5.3 Corpus: "*corpus*' que es la bolsa de palabras de gensim

6. Creacción del modelo

    6.1 Crear un modelo con 20 temas

    6.2 Utilice la instrucción "chunksize=100". ¿Qué efecto tiene?

7. Visualización. 

    7.1 Visualizar los resultados con la librería de *pyLDAvis*

    7.2 ¿Qué puede concluir de la visualización?

8. ¿Qué sucede si seleccionó 12, 15, 17 temas? Haga la visualización en estos casos. ¿Cuál es mejor?

### 1.Cargas Datos

In [5]:
df = pd.read_json(url)
df.head(5)

,content,target,target_names
2362,From: brown@ftms.UUCP (Vidiot)\nSubject: Re: p...,5,comp.windows.x
6854,From: markm@bigfoot.sps.mot.com (Mark Monninge...,7,rec.autos
11123,From: fmsalvat@eos.ncsu.edu (FRANK MICHAE SALV...,10,rec.sport.hockey
7235,From: erics@netcom.com (Eric Smith)\nSubject: ...,9,rec.sport.baseball
10824,From: avg@rodan.UU.NET (Vadim Antonov)\nSubjec...,11,sci.crypt


In [6]:
df.content.iloc[1]

"From: markm@bigfoot.sps.mot.com (Mark Monninger)\nSubject: Re: Auto air conditioning without Freon\nNntp-Posting-Host: 223.250.10.7\nReply-To: rapw20@email.sps.mot.com\nOrganization: SPS\nDistribution: usa\nLines: 20\n\nIn article <1993Apr15.222600.11690@research.nj.nec.com>  \nbehanna@syl.nj.nec.com (Chris BeHanna) writes:\n>  ...\n> \tSeveral chemists already have come up with several substitutes for\n> R12.  You don't hear about them because the Mobile Air Conditioning  \nSociety\n> (MACS), that is, the people who stand to rake in that $300 to $1000 per\n> retrofit per automobile, have mounted an organized campaign to squash  \nthose\n> R12 substitutes out of existence if not ban them altogether (on very  \nshaky\n> technical grounds, at best, on outright lies at worst).\n>  ...\n\nNow, I'm not saying you're wrong because I know that the R-12 substitutes  \nexist, but this sounds a lot like the 200mpg carbs that the oil companies  \nkeep us all from getting.\n\nMark\n\n"

2. Exploración de los datos y entendimiento del conjunto de datos

3. Interpretación del siguiente codigo
```
df.groupby(['target', 'target_names']).count()
```

**El siguiente codigo nos agrupa por la etiqueta y nos dice cuantas noticias hay por cada una de las etiquetas realizando un conteo.**

In [10]:
# Este codigo nos indica cuantas noticias hay en cada una de las etiquetas.
df.groupby(['target', 'target_names']).count()

,,content
target,target_names,
0,alt.atheism,92
1,comp.graphics,120
2,comp.os.ms-windows.misc,122
3,comp.sys.ibm.pc.hardware,138
4,comp.sys.mac.hardware,122
5,comp.windows.x,135
6,misc.forsale,145
7,rec.autos,137
8,rec.motorcycles,118


### 4. Para la Normalización utilizar ***spaCy*** y realizar las siguientes acciones en una sola función

In [36]:
def preprocessing(text):
    #Identificar el idioma
    idioma = detect(text)
    #4.1. Pasar las frases a minúsculas.
    text = text.lower()
    #4.6. Eliminar los emails  \S*@\S*\s
    #\S Identifica caracteres que no son espacio
    #* Lo realiza continuamente
    #\s Identifica espacios
    text = re.sub("\S*@\S*\s","",text)
    #print(text)
    #4.2. Eliminar los signos de puntuación.
    #4.7. Eliminar los saltos de línea
    #4.8. Eliminar las comillas simples
    text = re.sub("[^A-Za-z0-9]+"," ", text) 
    #print(text)
    # Tokenización
    if idioma == "en":
        doc = nlp_en(text)
    else:
        doc = nlp_es(text)
    #Realizamos una lista donde tenemos las palabras tokenizadas, su lema, su longitud
    # su tag y si son o no palabras de parada.
    resultado = [[tk.text, len(tk.text),tk.lemma_, tk.pos_, tk.is_alpha, tk.is_stop] for tk in doc]
    # 4.5. Eliminar las Stop-Words.
    resultado = [lista for lista in resultado if lista[5] == False]
    #4.3. Eliminar las palabras con menos de 3 carácteres.
    #4.4. Eliminar las palabras con mas de 12 carácteres.
    resultado = [lista for lista in resultado if (lista[1] <12) and (lista[1] > 3)]
    # 4.9. Filtrar las palabras que no sean Sustantivo, Adjetivo, Verbo o Adverbio.
    resultado = [lista for lista in resultado if (lista[3] == 'NOUN') or (lista[3] == 'ADJ') or (lista[3] == 'ADV') or (lista[3] == 'VERB')]
    #4.10. Pasar la palabra a su lema
    #Tomamos los lemas de cada una de las palabras de la lista despues de la limpieza.
    result = []
    for i in resultado:
        result.append(i[2])
        
    return result


In [37]:
a = "From: markm@bigfoot.sps.mot.com (Mark Monninger)\nSubject: Re: Auto air conditioning without Freon\nNntp-Posting-Host: 223.250.10.7\nReply-To: rapw20@email.sps.mot.com\nOrganization: SPS\nDistribution: usa\nLines: 20\n\nIn article <1993Apr15.222600.11690@research.nj.nec.com>  \nbehanna@syl.nj.nec.com (Chris BeHanna) writes:\n>  ...\n> \tSeveral chemists already have come up with several substitutes for\n> R12.  You don't hear about them because the Mobile Air Conditioning  \nSociety\n> (MACS), that is, the people who stand to rake in that $300 to $1000 per\n> retrofit per automobile, have mounted an organized campaign to squash  \nthose\n> R12 substitutes out of existence if not ban them altogether (on very  \nshaky\n> technical grounds, at best, on outright lies at worst).\n>  ...\n\nNow, I'm not saying you're wrong because I know that the R-12 substitutes  \nexist, but this sounds a lot like the 200mpg carbs that the oil companies  \nkeep us all from getting.\n\nMark\n\n"
k = preprocessing(a)
print(k)

['subject', 'auto', 'freon', 'nntp', 'post', 'host', 'reply', 'line', 'article', 'write', 'chemist', 'come', 'substitute', 'hear', 'mobile', 'society', 'mac', 'people', 'stand', 'rake', 'retrofit', 'automobile', 'mount', 'organize', 'campaign', 'squash', 'substitute', 'existence', 'altogether', 'shaky', 'technical', 'ground', 'good', 'outright', 'lie', 'bad', 'say', 'wrong', 'know', 'substitute', 'exist', 'sound', 'carb', 'company', 'get', 'mark']


# PUNTO 2

- Con los datos normalizados y vectorizados aplique el método LSI. Compare los resultados.

# PUNTO 3

- Con los datos normalizados y vectorizados. ¿Es posible aplicar k-means?

- En caso de una respuesta afirmativa. Utilice alguno el método del codo, el método de la silueta, y el método GAP para determinar cuáles valores de $k$ (número de clusters) son sugeridos y aplique el modelo.

- Comparar resultados con los métodos anteriores: k-means, LDA y LSI.

- Concluya